In [2]:
import glob
import json
import numpy.random as random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce

## question

In [3]:
question = glob.glob('../../../../ICDAR-2019-SROIE/data/box/*')

In [4]:
txt_list = []
for text in question:
    with open(text) as f:
        txt = f.read()
    txt_list.append(txt)

In [5]:
def split_only_8(string):
    comma_split_string = [elem for elem in string.split(',') if elem] 
    element_8_list = []
    leftover = []
    for idx, txt in enumerate(comma_split_string):
        if idx < 8:
            element_8_list.append(int(txt))
        else:
            leftover.append(txt)
    element_8_list.append(','.join(leftover))
    return element_8_list

In [6]:
corpus = [[split_only_8(row) for row in text.split('\n')] for text in txt_list]

In [7]:
df_list = [pd.DataFrame(c, columns=['x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4','y4','transcript']).dropna() for c in corpus]

In [8]:
df_long = reduce(lambda a,b: pd.concat([a, b]), df_list)

In [9]:
text_list = list(df_long.transcript)

## answer

In [10]:
answer = glob.glob('../../../../ICDAR-2019-SROIE/data/key/*')

In [11]:
json_list = []
for ans in answer:
    with open(ans) as f:
        json_list.append(json.load(f))

In [12]:
json_list[1]

{'company': 'INDAH GIFT & HOME DECO',
 'date': '19/10/2018',
 'address': '27, JALAN DEDAP 13, TAMAN JOHOR JAYA, 81100 JOHOR BAHRU, JOHOR.',
 'total': '60.30'}

In [13]:
answer_df = pd.DataFrame(json_list)

In [14]:
set_company = list(answer_df.company)

In [15]:
set_address = list(answer_df.address)

In [16]:
set_date = list(answer_df.date)

In [17]:
set_total = list(answer_df.total)

In [18]:
not_none = ' '.join(list(map(str, answer_df.company + answer_df.address + answer_df.date)))

In [19]:
none = list(filter(lambda e: e not in not_none, text_list))

In [20]:
set_none = random.choice(none, 500)

In [21]:
len(set_company), len(set_date), len(set_address), len(set_total), len(set_none)

(626, 626, 626, 626, 500)

In [22]:
answer = {0:'company', 1:'date', 2:'address', 3:'None'}
answer_inv = {value:key for key, value in answer.items()}

In [23]:
company_label_list = [(_, answer_inv['company']) for _ in set_company]
date_label_list = [(_, answer_inv['date']) for _ in set_date]
address_label_list = [(_, answer_inv['address']) for _ in set_address]
none_label_list = [(_, answer_inv['None']) for _ in set_none]

In [24]:
corpus_label = company_label_list + date_label_list + address_label_list + none_label_list

In [25]:
from glob import glob
import sys
sys.path.append('../../../modules/database/')
from db_util import *
from db import *

In [26]:
path = r'C:\Users\hscho\Desktop\long8v/DB/'
name = 'word_classificaton_data'
description = 'data for word classification, no splitted'
db_raw = create_data_db(path=path, name=name, description=description, data_class='RAW')

In [27]:
cache = {}
for index, value in tqdm(enumerate(corpus_label)):
    cache = update_cache(cache, index=index, label=str(value[1]), image=None, text=str(value[0]), ref=None)

2378it [00:00, 140280.09it/s]


In [28]:
write_cache_to_db(db_raw, db_raw.open_db(b'db_data'), cache)

In [29]:
print_env(db_raw)

+-------------+----------------------------------------------------------+
| Key         | Value                                                    |
+-------------+----------------------------------------------------------+
| class       | RAW                                                      |
| created     | 2020-09-09 14:21:27                                      |
| db_data     | (database)                                               |
| description | data for word classification, no splitted                |
| name        | word_classificaton_data                                  |
| note        |                                                          |
| reference   | C:\Users\hscho\Desktop\long8v/DB/word_classificaton_data |
| sep         | 	                                                        |
| updated     | 2020-09-09 14:21:27                                      |
+-------------+----------------------------------------------------------+


In [30]:
register_db(db=db_raw)

## tokenizer

In [31]:
from tensorflow.keras import preprocessing

In [47]:
all_text = list(zip(*corpus_label))[0]
all_text_chr = list(map(lambda e: list(str(e)), all_text))
tokenizer = preprocessing.text.Tokenizer(oov_token='<UNK>')
tokenizer.fit_on_texts(all_text_chr)

In [53]:
tokenizer_path = r'C:\Users\hscho\Desktop\long8v/DB/'
with open('{}/tokenizer.json'.format(tokenizer_path), 'w') as f:
    json.dump(tokenizer.to_json(), f)